<a href="https://colab.research.google.com/github/HLaw18/4105/blob/main/HW7_4105.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import time

In [ ]:
#1a
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

start_time = time.time()
history = model.fit(train_images, train_labels, epochs=300, validation_data=(test_images, test_labels))
end_time = time.time()

print(f"Training Time: {end_time - start_time} seconds")
training_loss = history.history['loss'][-1]
evaluation_accuracy = model.evaluate(test_images, test_labels)[1]
print(f"Training Loss: {training_loss}")
print(f"Evaluation Accuracy: {evaluation_accuracy}")

170498071/170498071 [==============================] - 5s 0us/step
Epoch 1/300
1563/1563 [==============================] - 20s 6ms/step - loss: 1.4693 - accuracy: 0.4648 - val_loss: 1.1888 - val_accuracy: 0.5683
Epoch 2/300
1563/1563 [==============================] - 9s 6ms/step - loss: 1.1116 - accuracy: 0.6060 - val_loss: 1.0577 - val_accuracy: 0.6220
Epoch 3/300
1563/1563 [==============================] - 8s 5ms/step - loss: 0.9508 - accuracy: 0.6664 - val_loss: 0.9267 - val_accuracy: 0.6750
Epoch 4/300
1563/1563 [==============================] - 9s 6ms/step - loss: 0.8473 - accuracy: 0.7035 - val_loss: 0.8887 - val_accuracy: 0.6901
Epoch 5/300
1563/1563 [==============================] - 9s 6ms/step - loss: 0.7640 - accuracy: 0.7315 - val_loss: 0.8277 - val_accuracy: 0.7126
Epoch 6/300
1563/1563 [==============================] - 9s 6ms/step - loss: 0.7008 - accuracy: 0.7545 - val_loss: 0.8666 - val_accuracy: 0.7057
Epoch 7/300
1563/1563 [==============================] - 9s 6m

In [ ]:
#1b
model_extended = models.Sequential()
model_extended.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model_extended.add(layers.MaxPooling2D((2, 2)))
model_extended.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_extended.add(layers.MaxPooling2D((2, 2)))
model_extended.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_extended.add(layers.MaxPooling2D((2, 2)))
model_extended.add(layers.Flatten())
model_extended.add(layers.Dense(256, activation='relu'))
model_extended.add(layers.Dense(10, activation='softmax'))

model_extended.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

start_time_extended = time.time()
history_extended = model_extended.fit(train_images, train_labels, epochs=300, validation_data=(test_images, test_labels))
end_time_extended = time.time()

print(f"Extended Model Training Time: {end_time_extended - start_time_extended} seconds")
print(f"Extended Model Training Loss: {history_extended.history['loss'][-1]}")
print(f"Extended Model Evaluation Accuracy: {model_extended.evaluate(test_images, test_labels)[1]}")

Epoch 1/300
1563/1563 [==============================] - 12s 7ms/step - loss: 1.4998 - accuracy: 0.4525 - val_loss: 1.3122 - val_accuracy: 0.5329
Epoch 2/300
1563/1563 [==============================] - 11s 7ms/step - loss: 1.1208 - accuracy: 0.6047 - val_loss: 1.0437 - val_accuracy: 0.6308
Epoch 3/300
1563/1563 [==============================] - 11s 7ms/step - loss: 0.9555 - accuracy: 0.6629 - val_loss: 1.0140 - val_accuracy: 0.6447
Epoch 4/300
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8456 - accuracy: 0.7027 - val_loss: 0.9392 - val_accuracy: 0.6798
Epoch 5/300
1563/1563 [==============================] - 9s 6ms/step - loss: 0.7581 - accuracy: 0.7341 - val_loss: 0.8808 - val_accuracy: 0.7050
Epoch 6/300
1563/1563 [==============================] - 9s 6ms/step - loss: 0.6888 - accuracy: 0.7578 - val_loss: 0.8410 - val_accuracy: 0.7142
Epoch 7/300
1563/1563 [==============================] - 9s 5ms/step - loss: 0.6213 - accuracy: 0.7830 - val_loss: 0.8417 - val

In [ ]:
#2
class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(x)
        out = self.relu(out)
        return out

class ResNet10(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet10, self).__init__()
        self.in_channels = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self.make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self.make_layer(block, 64, num_blocks[2], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

def train(model, criterion, optimizer, epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion.to(device)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}")

def test(model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Accuracy on test set: {accuracy * 100:.2f}%")

resnet10 = ResNet10(ResNetBlock, [1, 1, 1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet10.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)

train(resnet10, criterion, optimizer, epochs=300)
test(resnet10)

resnet10_weight_decay = ResNet10(ResNetBlock, [1, 1, 1])
optimizer_weight_decay = optim.SGD(resnet10_weight_decay.parameters(), lr=0.01, momentum=0.9, weight_decay=0.001)

train(resnet10_weight_decay, criterion, optimizer_weight_decay, epochs=300)
test(resnet10_weight_decay)

resnet10_dropout = ResNet10(ResNetBlock, [1, 1, 1])
resnet10_dropout.fc = nn.Sequential(
    nn.Linear(64, 256),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(256, 10)
)
optimizer_dropout = optim.SGD(resnet10_dropout.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)

train(resnet10_dropout, criterion, optimizer_dropout, epochs=300)
test(resnet10_dropout)

resnet10_bn = ResNet10(ResNetBlock, [1, 1, 1])
resnet10_bn.fc = nn.Sequential(
    nn.Linear(64, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

100%|██████████| 170498071/170498071 [00:10<00:00, 16341414.86it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified
Epoch 1/300, Loss: 1.59210804203892
Epoch 2/300, Loss: 1.21388302061259
Epoch 3/300, Loss: 1.0595552994467108
Epoch 4/300, Loss: 0.964470315162483
Epoch 5/300, Loss: 0.9035164424982827
Epoch 6/300, Loss: 0.8421143467926309
Epoch 7/300, Loss: 0.804350350366529
Epoch 8/300, Loss: 0.7669548871343398
Epoch 9/300, Loss: 0.7314508683083917
Epoch 10/300, Loss: 0.7036980501068827
Epoch 11/300, Loss: 0.6810534130734252
Epoch 12/300, Loss: 0.6589728748935568
Epoch 13/300, Loss: 0.6407483408365713
Epoch 14/300, Loss: 0.6236408146293572
Epoch 15/300, Loss: 0.6128356570325544
Epoch 16/300, Loss: 0.5953512428437963
Epoch 17/300, Loss: 0.5817578385591202
Epoch 18/300, Loss: 0.5687048468748321
Epoch 19/300, Loss: 0.5636324975115564
Epoch 20/300, Loss: 0.5534984634622283
Epoch 21/300, Loss: 0.5390841603812659
Epoch 22/300, Loss: 0.5343528678045248
Epoch 23/300, Loss: 0.52148441737875
Epoch 24/300, Loss: 0.5154637404338783
Epoch 25/300, Loss: 0.5084638459336422
Epoc